In [461]:
#import packages
import pandas as pd
import requests
from newsapi.newsapi_client import NewsApiClient
from pandas.io.json import json_normalize

In [462]:
#Need more news articles, could filter based on keywords later in text processing and sentiment analysis
#current results are very noisy and contain articles not related to the fashion industry
# Init
newsapi = NewsApiClient(api_key='59a64b0ed01c48149c842c144e2004bb')

sources = newsapi.get_sources()
all_articles = newsapi.get_everything(q='(H&M) AND (sustainable OR slavery OR ecological OR underpaid)',
                                    from_param='2019-10-03',
                                      to='2019-11-03',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)
articles= all_articles

articles= articles['articles']


In [463]:
df_articles= json_normalize(articles)
df_articles=df_articles[['title', 'content']]

df_articles

,title,content
0,H&M CEO Sees ‘Terrible’ Fallout as Consumer Sh...,(Bloomberg) -- The chief executive officer of ...
1,Reduced body sizes in climate-impacted Borneo ...,"<li>1.Chen, I.-C., Hill, J. K., Ohlemuller, R...."
2,Quantifying the impacts of defaunation on natu...,"<li>1.Watson, J. E. M. et al. The exceptional ..."
3,Sustainability Finally Takes Centre Stage in S...,"SHANGHAI, China — From whichever way you exami..."
4,Pesticide impacts through aquatic food webs,None
5,[ASAP] Direct Synthesis of Cycloalkanes from D...,Cycloalkanes are ubiquitous structural motifs ...
6,Artificial intelligence reveals environmental ...,Deep learning predicts elevational ranges\r\nT...
7,Green oxidation of indoles using halide catalysis,Oxidative rearrangement of tetrahydro--carboli...
8,I Rented Clothes for a Month Instead of Buying...,As someone who spends an average of $300 to $5...
9,Earthworms' place on Earth,None


In [464]:
#text preprocessing
def cleanTxt(txt):
    """Returns dataframe with cleaned, but not joined strings
    (the structure of dataframe remains the same)"""
    txt = pd.DataFrame(txt)
    txt.fillna("", inplace = True)
 # replacing of symbols, Umlaute, and other uncool signs, 
 #furthermore some common words are abbreviated
    dictionarysymbols1 = {'\-':'','"':'','\(': '','\)':'', '\<':'', '\>':'', '\+':'',
                          '\.':'', '\,':'', '&':'', '\'':'', '\’':'',
                          '–':'', '/':' ', '/':' ', '//':' ', '\r':' ', '\n':' ' }
    dictionarysymbols2 = {'NA':'', 'NaN':'', '%':'', '~':'','\[':'', '\]':'', '$':'',
                          '£':'', '\_':'', '\:':'','\;':'','!':'', 'ß':'ss'}
    twoLetterWords = {r'\b\w{1,2}\b':''} #removes two letter words
    txt = txt.apply(lambda col: col.str.replace('İ', 'I').str.lower())
    txt = txt.replace(dictionarysymbols1, regex = True)
    txt = txt.replace(dictionarysymbols2, regex = True)
    txt = txt.apply(lambda x: x.str.strip()) #strip leading and training spaces
    #txt = txt.str.replace('  ', '') #remove double spaces after initial processing
    return txt

def concatenate_words(txt):
    """Concatenate words in clean dataframe, returns Series containing lists of words"""
    txt = txt.apply(lambda x: " ".join(x), axis=1) #concatenate all columns
    txt = txt.str.replace('nan', '') #remove nan strings after initial preprocessing
    #txt = txt.apply(lambda x: list(set([y for y in x.split() if len(y) > 2])))# remove duplicates - simpler version
    # we don't return words separated by space because we need both versions: with lists and with space-separated words
    # txt= (txt.str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' ')) #remove duplicate strings, keep only unique values
    return txt

In [476]:
df= concatenate_words(cleanTxt(df_articles))

In [477]:
# very basic textblob sentiment analysis
#could break down articles into sentences and then measure polarization, might get better/moreaccurate results
#articles need to be filtered because current results are very noisy. 
#could be significantly improved with NER, tokenization, lemmatization, POS tagging etc, tfidf 
from textblob import TextBlob

score= []   

for n in range(len(df)):
    blob=TextBlob(df[n]).sentiment.polarity
    score.append(blob)
    


In [474]:
#combine score and text columns
df= pd.DataFrame(df,columns=['text'])
df['score'] = score
display(df)

#get mean score TextBlobs polarity scores from -1 to 1
df['score'].mean()

,text,score
0,hm ceo sees ‘terrible fallout as consumer sham...,-0.316667
1,reduced body sizes in climateimpacted borneo m...,0.280000
2,quantifying the impacts of defaunation on natu...,0.225758
3,sustainability finally takes centre stage in s...,0.033333
4,pesticide impacts through aquatic food webs,0.000000
5,asap direct synthesis of cycloalkanes from dio...,-0.087500
6,artificial intelligence reveals environmental ...,-0.110417
7,green oxidation of indoles using halide cataly...,-0.016667
8,i rented clothes for a month instead of buying...,0.103283
9,earthworms place on earth,0.000000


In [440]:
#information from the various data sources would be imported to common database
barcode= '01073139284309901025870724020191021110000004221'
data= pd.DataFrame(columns=['Company', 'Barcode', 'Labor', 'Environment', 'Community', 'Overall'])
data.to_csv('NewsData.csv')